In [113]:
#Proyecto 1 de:
#Gómez Guzmán Luis Enrique

In [116]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount("mnt")


class Grafica():
    """Clase para representar la gráfica simple usando la matriz
    de adyacencia"""
    def __init__(self, nodos, arcos):
        """Constructor que inicializa la matriz, nodos y arcos"""
        self.matriz = np.zeros((len(nodos), len(nodos)))
        for x in arcos:
            self.matriz[x[0], x[1]] = 1

    def __str__(self):

        return str(self.matriz)

    def agregar_arco(self, tupla):
        """Agrega un arco entre los nodos de la tupla"""
        self.matriz[tupla[0], tupla[1]] = 1

    def quitar_arco(self, tupla):
        """Elimina el arco entre los nodos de la tupla"""
        self.matriz[tupla[0], tupla[1]] = 0

    def num_nodos(self):
        """Devuelve el número de nodos en la gráfica"""
        return len(self.matriz)

    def num_arcos(self):
        """Devuelve el número total de arcos en la gráfica"""
        return int(self.matriz.sum())

    def agregar_nodo(self):
        """Agrega un nodo a la gráfica"""
        vertical = np.zeros((len(self.matriz), 1))
        horizontal = np.zeros((1, len(self.matriz) + 1))
        self.matriz = np.vstack((np.hstack((self.matriz, vertical)),
                                 horizontal))

    def quitar_nodo(self, nodo):
        """Elimina un nodo de la gráfica"""
        self.matriz = np.delete(self.matriz, (nodo), axis=0)
        self.matriz = np.delete(self.matriz, (nodo), axis=1)


class Red(Grafica):
    """Clase que hereda de Grafica y maneja pesos en los arcos"""

    def __init__(self, nodos, arcos, pesos = None):
        """Constructor de la clase hija (Red)"""

        super().__init__(nodos, arcos)  # Llamamos al constructor de
        #la clase padre
        self.pesos = np.zeros_like(self.matriz)  # Inicializamos la
        #matriz de pesos

        # Si se proporcionan pesos, los asignamos a la matriz de pesos
        if pesos is not None:
            for i, arco in enumerate(arcos):
                if i < len(pesos):
                    self.pesos[arco[0], arco[1]] = pesos[i]

    def agregar_peso(self, tupla, peso):
        """Agregamos el peso del arco"""
        if self.matriz[tupla[0], tupla[1]] == 1:
            self.pesos[tupla[0], tupla[1]] = peso
        else:
            print("El arco no existe")

    def obtener_peso(self, tupla):
        """Obtenemos el peso del arco"""
        return self.pesos[tupla[0], tupla[1]]

    def calcular_importancia(self, pi_inicial):
        """Método para calcular la importancia de las páginas usando el
        algoritmo PageRank"""

        foo = np.zeros_like(self.matriz)
        for i in range(len(self.matriz)):
            suma_fila = self.matriz[i, :].sum()
            if suma_fila > 0:
                foo[i, :] = self.matriz[i, :] / suma_fila
            else:
                foo[i, :] = np.ones(len(self.matriz)) / len(self.matriz)

        pi = pi_inicial
        pi_1 = pi@foo

        while np.linalg.norm(pi - pi_1) > 1e-6:
            pi = pi_1
            pi_1 = pi@foo

        return pi

    def calcular_importancia_con_d(self, pi_inicial, d):
        """Método para calcular la importancia con d"""
        # Creacion de  las matrices U y P
        n = len(self.matriz)
        P = np.zeros_like(self.matriz, dtype = float)
        U = np.ones((n, n)) / n

        # matriz P
        for i in range(n):
            suma_fila = self.matriz[i, :].sum()
            if suma_fila > 0:
                P[i, :] = self.matriz[i, :] / suma_fila
            else:
                P[i, :] = np.ones(n) / n

        # Calcular la nueva matreiz siguiendo la eucacion
        # π_m+1 = π_m*(dP + (1−d)U)
        matriz_modelo = d * P + (1 - d) * U

        # Inicializar pi
        pi = pi_inicial.copy()
        #utilizamos .copy para no alterar el valor de pi_inicial
        while True:
            pi_nuevo = pi@matriz_modelo
            if np.allclose(pi, pi_nuevo):
              #usamos allclose para comparar los pi
                break
            pi = pi_nuevo / np.sum(pi_nuevo)

        return pi

def construccion_pd(ruta_nombre_del_archivo):
    """leemos el excel y creamos una gráfica de Red
    con los nodos, arcos y pesos"""

    df = pd.read_excel(ruta_nombre_del_archivo)
    Nodos = list(range(len(df["Index"])))
    x = list(df["Cited by"])
    Arcos = []
    for i in range(len(x)):
        for j in x[i].split(','):
            Arcos.append((int(j) - 1, i))
    return Red(Nodos, Arcos)


#implementación del metodo main

def main():
    # Ruta del excel
    ruta = "/content/mnt/MyDrive/Web.xlsx"

    # Construimos la red usando el archivo
    red = construccion_pd(ruta)

    #Inciso i)
    pi_inicial = np.ones((1, len(red.matriz))) / len(red.matriz)
    pi = red.calcular_importancia(pi_inicial)
    valor_maximo = np.argmax(pi) + 1
    print("La página más importante tiene el índice:", valor_maximo)

    # inciso ii)
    pi_inicial = np.ones((1, len(red.matriz))) / 10
    pi = red.calcular_importancia(pi_inicial)
    val_maximo = np.argmax(pi) + 1
    print("El vector pi, dado que tomamos las páginas .ru es:", pi)
    print("La página más importante tiene el índice:", val_maximo)

    # inciso iii)
    print("\n- Resultados claculados con la ecuación iterativa-")
    pi_inicial = np.ones((1, len(red.matriz))) / len(red.matriz)

    for d in [0.5, 0.85, 1.0]:
        print(f"\nResultados para d = {d}:")
        pi = red.calcular_importancia_con_d(pi_inicial, d)
        # Mostramos las 5
        #paginas más importantes para el análisis de datos

        indices_top = np.argsort(pi[0])[-5:][::-1]
        #Usamos argsort para mostrar los valores ma altos
        print("\nLas 5 páginas más importantes son:")
        for pos, idx in enumerate(indices_top, 1):
            print(f"{pos}. Página {idx + 1}: {pi[0][idx]:.6f}")


if __name__ == "__main__":
    main()

Drive already mounted at mnt; to attempt to forcibly remount, call drive.mount("mnt", force_remount=True).
La página más importante tiene el índice: 26
El vector pi, dado que tomamos las páginas .ru es: [[7.18540019e-07 9.97130812e-07 6.20601531e-07 1.33342858e-06
  5.23924237e-07 8.04120514e-07 1.34244055e-06 8.17119173e-07
  1.07948520e-06 9.41322368e-07 4.94332410e-07 5.13479355e-07
  7.38978606e-07 7.42408074e-07 9.53772899e-07 9.37500000e-02
  9.37500000e-02 9.37500000e-02 9.37500000e-02 1.25000000e-01
  9.37500000e-02 9.37500000e-02 9.37500000e-02 1.25000000e-01
  9.37500000e-02 1.59998738e+00]]
La página más importante tiene el índice: 26

- Resultados claculados con la ecuación iterativa-

Resultados para d = 0.5:

Las 5 páginas más importantes son:
1. Página 26: 0.073999
2. Página 4: 0.047021
3. Página 24: 0.043855
4. Página 20: 0.043845
5. Página 7: 0.043385

Resultados para d = 0.85:

Las 5 páginas más importantes son:
1. Página 26: 0.200004
2. Página 24: 0.046882
3. Página 